In [ ]:
import pandas as pd
from plotnine import ggplot, aes, geom_line, xlab, ylab, scale_color_discrete, facet_wrap, theme_minimal, element_text, theme

In [ ]:
detailed_df = pd.read_csv('../acc_rate_csvs/detailed_mlp_condensed.csv')
detailed_df['dataset'] = 'detailed'

systematic_df = pd.read_csv('../acc_rate_csvs/systematic_mlp_condensed.csv')
systematic_df['dataset'] = 'systematic'

final_df = pd.concat([detailed_df, systematic_df])
final_df['layer_size'] = pd.Categorical(final_df['layer_size'], ordered=True)

In [ ]:
# Plotting
plot_combined = (
    ggplot(final_df, aes(x='layer_size', y='acc', color='factor(n_layer)')) + 
    geom_line() +
    aes(group='factor(n_layer)') +
    xlab("number of neurons") +
    ylab("validation accuracy") +
    scale_color_discrete(name="Number of \nHidden Layers") +
    facet_wrap('~dataset', ncol=3, scales='free_y') +
    theme_minimal() +
    theme(figure_size=(10, 3), text=element_text(size=12))
)

print(plot_combined)

In [ ]:
plot_combined.save('mlp.pdf')
plot_combined.save('mlp.jpg')